In [3]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms

train_data = torchvision.datasets.MNIST(root="./data", train=True, download=True,
                                    transform=torchvision.transforms.Compose([
                                        torchvision.transforms.ToTensor(),  # 转换成张量
                                    ]))
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64)

In [5]:
import torch
from denoising_diffusion_pytorch import Unet, GaussianDiffusion, Trainer

device = "cuda" if torch.cuda.is_available() else "cpu"
model = Unet(
    dim = 64,
    channels = 1,
    dim_mults = (1, 2, 2)
)
model.to(device)
diffusion = GaussianDiffusion(
    model,
    objective = 'pred_noise',
    image_size = 28,
    timesteps = 500    # number of steps
)
diffusion.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

epochs = 10
for epoch in range(epochs):
    print('Epoch: ', epoch+1)
    for step, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        
        batch_size = images.shape[0]
        images = images.to(device)
        loss = diffusion(images)

        if step % 100 == 0:
            print("Step: ", step+1, "Loss:", loss.item())
            print(loss.device)
            
        loss.backward()
        optimizer.step()
torch.save(model.state_dict(),'test.pt')

Epoch:  1
Step:  1 Loss: 1.0706336498260498
cpu


KeyboardInterrupt: 

In [6]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from denoising_diffusion_pytorch import Unet, GaussianDiffusion, Trainer
device = "cuda" if torch.cuda.is_available() else "cpu"

model = Unet(
    dim = 64,
    channels = 1,
    dim_mults = (1, 2, 2)
)
model.load_state_dict(torch.load('test.pt'))
model.to(device)
diffusion = GaussianDiffusion(
    model,
    objective = 'pred_noise',
    image_size = 28,
    timesteps = 500    # number of steps
)
diffusion.to(device)

generated_images = diffusion.sample(batch_size=64)
# DDPM生成MNIST新数据图片
fig = plt.figure(figsize=(16, 16), constrained_layout=True)
gs = fig.add_gridspec(8, 8)

imgs = generated_images.reshape(8, 8, 28, 28).to('cpu')
for n_row in range(8):
    for n_col in range(8):
        f_ax = fig.add_subplot(gs[n_row, n_col])
        f_ax.imshow((imgs[n_row, n_col]), cmap="gray")
        f_ax.axis("off")

FileNotFoundError: [Errno 2] No such file or directory: 'test.pt'